<a href="https://colab.research.google.com/github/ykitaguchi77/statistics_for_articles/blob/main/Blepharoptosis_dryeye_eyedrop_20250521.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: gdrive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# -*- coding: utf-8 -*-
"""眼瞼下垂術後SPK_総合解析_フル.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1_example_notebook_id # このIDはダミーです。実際のIDに置き換えてください。

# 眼瞼下垂術後SPKと点眼薬・BUTに関する総合解析

**目的:**
眼瞼下垂手術後の角膜SPKの発生・改善に関連する要因を多角的に調査する。
主な焦点として、以下の2点を検証する。
1. 点眼薬の使用が、SPKの改善や悪化防止（予防）に直接的に関連しているか (Fisherの正確確率検定)。
2. 「術後点眼薬の使用が術後BUT（涙液層破壊時間）を改善させ、その結果として角膜SPKの改善に寄与するのではないか」という媒介効果の仮説 (ブートストラップ媒介分析)。

**解析ステップ:**
1. データ準備: 必要なライブラリのインポート、データ読み込み、前処理
2. 点眼薬使用とSPK変化の直接的関連の検証 (Fisherの正確確率検定)
   - SPK改善 (術前SPK(+) -> 術後SPK(-)) と点眼薬の関連
   - SPK悪化/予防 (術前SPK(-) -> 術後SPK(+) or (-)) と点眼薬の関連
3. 術後点眼薬 (X) と ΔBUT (M) の関連性の確認 (媒介分析のパス a)
4. ΔBUT (M) と SPK改善 (Y) の関連性の確認 (媒介分析のパス b)
5. 術後点眼薬 (X) と SPK改善 (Y) の直接効果の確認 (媒介分析のパス c')
6. ブートストラップ法による正式な媒介分析: 間接効果 (a*b) の推定と信頼区間の算出 (SPK改善モデルについて)

**使用データ:**
事前に提供された`眼瞼下垂★ドライアイ_kit (1).xlsx` の `all` シートのデータを使用します。
**このノートブックを実行する前に、上記ExcelファイルをColab環境にアップロードするか、Google Driveに配置してマウントし、ファイルパスを適切に設定してください。**
"""

# =========================================
# Section 1: データ準備
# =========================================
# 目的: 解析に必要なライブラリをインポートし、データを読み込み、分析に適した形に前処理する。
# -----------------------------------------

import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import mannwhitneyu, fisher_exact
from tqdm import tqdm # ブートストラップの進捗表示用

# Google Colabにファイルをアップロードした場合のサンプルコード
# from google.colab import files
# print("Excelファイルをアップロードしてください:")
# uploaded = files.upload()
# if uploaded:
#     file_path = next(iter(uploaded))
#     print(f"\nファイル '{file_path}' がアップロードされました。")
# else:
#     print("ファイルがアップロードされませんでした。処理を中断します。")
#     file_path = None

# ローカル環境またはマウントしたGoogle Driveのファイルパス (適宜変更してください)
file_path = "/content/drive/Shareddrives/岩崎Dr_IgG4 deulk/眼瞼下垂ドライアイ/眼瞼下垂★ドライアイ_kit.xlsx" # ここにご自身のファイルパスを指定

# データの読み込み
try:
    #df_all_original = pd.read_excel(file_path, sheet_name="対象症例sides", header=0)
    df_all_original = pd.read_excel(file_path, sheet_name="all", header=1)
    print(f"ファイル '{file_path}' を正常に読み込みました。")
except FileNotFoundError:
    print(f"エラー: ファイル '{file_path}' が見つかりません。")
    df_all_original = pd.DataFrame()
except Exception as e:
    print(f"ファイルの読み込み中に予期せぬエラーが発生しました: {e}")
    df_all_original = pd.DataFrame()


if not df_all_original.empty:
    cols_to_use = ["SPK pre", "SPK post", "BUT pre", "BUT post", "age",
                   "点眼薬 pre", "点眼薬 post", "levator_function pre", "ΔMRD-1"]
    missing_cols = [col for col in cols_to_use if col not in df_all_original.columns]
    if missing_cols:
        print(f"エラー: 必要な列が見つかりません: {missing_cols}")
        df = pd.DataFrame()
    else:
        df = df_all_original[cols_to_use].copy()
        cols_to_convert_numeric = ["SPK pre", "SPK post", "BUT pre", "BUT post", "age",
                                   "levator_function pre", "ΔMRD-1"]
        for col in cols_to_convert_numeric:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        # 点眼薬変数の定義 (eye_pre, eye_post)
        df["点眼薬 pre"] = pd.to_numeric(df["点眼薬 pre"], errors='coerce')
        df["点眼薬 post"] = pd.to_numeric(df["点眼薬 post"], errors='coerce')
        df["eye_pre"] = (df["点眼薬 pre"].fillna(0) > 0).astype(int)
        df["eye_post"] = (df["点眼薬 post"].fillna(0) > 0).astype(int)
        # eye_pre: 術前点眼薬の使用有無 (0=非使用, 1=使用)
        # eye_post: 術後点眼薬の使用有無 (0=非使用, 1=使用)

        df["ΔBUT"] = df["BUT post"] - df["BUT pre"]

        # SPK改善の定義 (SPK pre=1 & SPK post=0 -> improved=1)
        df_spk_pre1 = df[df["SPK pre"] == 1].copy()
        if not df_spk_pre1.empty:
            df_spk_pre1["improved"] = (df_spk_pre1["SPK post"] == 0).astype(int)
        else:
            df_spk_pre1["improved"] = pd.Series(dtype=int)


        # SPK悪化/予防の定義 (SPK pre=0 の症例について)
        #   worsened=1 (悪化): SPK pre=0 & SPK post=1
        #   worsened=0 (予防成功/変化なし): SPK pre=0 & SPK post=0
        df_spk_pre0 = df[df["SPK pre"] == 0].copy()
        if not df_spk_pre0.empty:
            df_spk_pre0["worsened"] = (df_spk_pre0["SPK post"] == 1).astype(int)
        else:
            df_spk_pre0["worsened"] = pd.Series(dtype=int)


        print("\nデータ準備完了。")
        if not df_spk_pre1.empty :
            print(f"SPK改善モデルの対象症例数 (SPK pre=1): {len(df_spk_pre1)}")
            print(f"  うちSPK改善 (improved=1) した症例数: {df_spk_pre1['improved'].sum()}")
        if not df_spk_pre0.empty:
            print(f"SPK悪化/予防モデルの対象症例数 (SPK pre=0): {len(df_spk_pre0)}")
            print(f"  うちSPK悪化 (worsened=1) した症例数: {df_spk_pre0['worsened'].sum()}")

        covariates = ["age", "BUT pre", "levator_function pre", "ΔMRD-1", "eye_pre"]
else:
    print("データファイルの読み込みに失敗したため、これ以降の処理は実行されません。")
    df = pd.DataFrame()
    df_spk_pre1 = pd.DataFrame()
    df_spk_pre0 = pd.DataFrame()


# =============================================================================
# Section 2: 点眼薬使用とSPK変化の直接的関連の検証 (Fisherの正確確率検定)
# =============================================================================
# 目的: 点眼薬の使用（術前/術後）が、SPKの「改善」や「悪化防止（予防）」に
#       直接的に関連しているかを調べる。これは媒介効果を考慮しない単純な2群間比較。
# 手法: Fisherの正確確率検定 (2x2分割表)。
# -----------------------------------------------------------------------------
if not df.empty:
    print("\n\n--- Section 2: 点眼薬使用とSPK変化の直接的関連 (Fisher検定) ---")

    # --- 2.1 SPK改善 (術前SPK(+)だった症例) ---
    #     目的変数: improved (0=非改善, 1=改善)
    #     比較対象: eye_pre, eye_post
    print("\n--- 2.1 SPK改善 (術前SPK(+)症例) と点眼薬の関連 ---")
    if not df_spk_pre1.empty and 'improved' in df_spk_pre1.columns and not df_spk_pre1['improved'].dropna().empty :
        for eyedrop_var in ["eye_pre", "eye_post"]:
            # SPK post と 点眼薬 の両方に有効なデータを持つ症例のみを対象
            subset_fisher_imp = df_spk_pre1.dropna(subset=['SPK post', eyedrop_var])
            if len(subset_fisher_imp) > 0 and subset_fisher_imp['improved'].nunique() == 2 and subset_fisher_imp[eyedrop_var].nunique() == 2:
                contingency_table_imp = pd.crosstab(subset_fisher_imp[eyedrop_var], subset_fisher_imp['improved'])
                # 表の列名をわかりやすくする (0=非改善, 1=改善)
                contingency_table_imp.columns = ["Not Improved (SPK post=1)", "Improved (SPK post=0)"]
                # 表の行名をわかりやすくする (0=点眼なし, 1=点眼あり)
                contingency_table_imp.index = [f"{eyedrop_var} (No)", f"{eyedrop_var} (Yes)"]

                if contingency_table_imp.shape == (2,2) and contingency_table_imp.values.min() >=0 : # 2x2表であること、負の値がないことを確認
                    odds_ratio, p_value_fisher_imp = fisher_exact(contingency_table_imp)
                    print(f"\n{eyedrop_var} と SPK改善の関連:")
                    print(contingency_table_imp)
                    print(f"  Odds Ratio: {odds_ratio:.3f}, Fisher's Exact p-value: {p_value_fisher_imp:.4f}")
                    if p_value_fisher_imp < 0.05:
                        print(f"  解釈: {eyedrop_var} の使用とSPK改善には統計的に有意な関連が見られます。")
                        # 元の解析では eye_post で p=0.0225 だった
                    else:
                        print(f"  解釈: {eyedrop_var} の使用とSPK改善には統計的に有意な関連は見られませんでした。")
                else:
                    print(f"\n{eyedrop_var} と SPK改善の関連: 2x2分割表を作成できませんでした。データを確認してください。")
                    print(f"作成された表:\n{contingency_table_imp}")
            else:
                print(f"\n{eyedrop_var} と SPK改善の関連: 検定に必要なデータが不足しているか、変数のバリエーションがありません。")
    else:
        print("SPK改善モデルの対象データがありません。")


    # --- 2.2 SPK悪化/予防 (術前SPK(-)だった症例) ---
    #     目的変数: worsened (0=悪化なし/予防成功, 1=悪化)
    #     比較対象: eye_pre, eye_post
    print("\n--- 2.2 SPK悪化/予防 (術前SPK(-)症例) と点眼薬の関連 ---")
    if not df_spk_pre0.empty and 'worsened' in df_spk_pre0.columns and not df_spk_pre0['worsened'].dropna().empty:
        for eyedrop_var in ["eye_pre", "eye_post"]:
            subset_fisher_prev = df_spk_pre0.dropna(subset=['SPK post', eyedrop_var])
            if len(subset_fisher_prev) > 0 and subset_fisher_prev['worsened'].nunique() == 2 and subset_fisher_prev[eyedrop_var].nunique() == 2:
                contingency_table_prev = pd.crosstab(subset_fisher_prev[eyedrop_var], subset_fisher_prev['worsened'])
                # 表の列名をわかりやすくする (0=悪化なし, 1=悪化)
                contingency_table_prev.columns = ["Not Worsened (SPK post=0)", "Worsened (SPK post=1)"]
                # 表の行名をわかりやすくする (0=点眼なし, 1=点眼あり)
                contingency_table_prev.index = [f"{eyedrop_var} (No)", f"{eyedrop_var} (Yes)"]

                if contingency_table_prev.shape == (2,2) and contingency_table_prev.values.min() >=0 :
                    odds_ratio_prev, p_value_fisher_prev = fisher_exact(contingency_table_prev)
                    print(f"\n{eyedrop_var} と SPK悪化の関連:")
                    print(contingency_table_prev)
                    print(f"  Odds Ratio: {odds_ratio_prev:.3f}, Fisher's Exact p-value: {p_value_fisher_prev:.4f}")
                    if p_value_fisher_prev < 0.05:
                        print(f"  解釈: {eyedrop_var} の使用とSPK悪化（つまり、非使用が予防に関連）には統計的に有意な関連が見られます。")
                        # OR < 1 なら点眼使用群で悪化が少ない (予防効果)
                        # OR > 1 なら点眼使用群で悪化が多い
                    else:
                        print(f"  解釈: {eyedrop_var} の使用とSPK悪化（予防効果）には統計的に有意な関連は見られませんでした。")
                        # 元の解析では eye_pre (p=0.3331), eye_post (p=0.1205) ともに非有意だった
                else:
                    print(f"\n{eyedrop_var} と SPK悪化の関連: 2x2分割表を作成できませんでした。データを確認してください。")
                    print(f"作成された表:\n{contingency_table_prev}")
            else:
                print(f"\n{eyedrop_var} と SPK悪化の関連: 検定に必要なデータが不足しているか、変数のバリエーションがありません。")
    else:
        print("SPK悪化/予防モデルの対象データがありません。")

    print("\nFisher検定の解釈補足:")
    print("  - SPK改善モデルで eye_post のp値が有意であれば、「術後点眼はSPK改善と直接関連している」という根拠になります。")
    print("  - SPK悪化/予防モデルでp値が有意（かつOR<1で点眼あり群の悪化が少ない場合）であれば、「点眼はSPK発症予防と直接関連している」という根拠になります。")
    print("  これらの結果は、後の媒介分析とは独立した、直接的な関連性を示します。")

else:
    print("\n\n--- Section 2: スキップ (データ準備に失敗) ---")


# =============================================================================
# Section 3: 術後点眼薬 (X) と ΔBUT (M) の関連性の確認 (媒介分析のパス a)
# =============================================================================
# (前回のコードと同様のため、簡略化のためここでは省略しませんが、内容はほぼ同じです)
if not df.empty:
    delta_but_no_nan = df.dropna(subset=["ΔBUT", "eye_post"])
    group_eye_post_yes = delta_but_no_nan[delta_but_no_nan["eye_post"] == 1]["ΔBUT"]
    group_eye_post_no = delta_but_no_nan[delta_but_no_nan["eye_post"] == 0]["ΔBUT"]

    if not group_eye_post_yes.empty and not group_eye_post_no.empty:
        u_stat, p_value_mannwhitney_S3 = mannwhitneyu(group_eye_post_yes, group_eye_post_no, alternative='two-sided')
        print("\n\n--- Section 3: 術後点眼薬 (X) vs ΔBUT (M) (媒介分析パスaの準備) ---")
        print(f"術後点眼使用群のΔBUT中央値: {group_eye_post_yes.median():.2f} (N={len(group_eye_post_yes)})")
        print(f"術後点眼非使用群のΔBUT中央値: {group_eye_post_no.median():.2f} (N={len(group_eye_post_no)})")
        print(f"Mann-Whitney U検定 p値: {p_value_mannwhitney_S3:.4f}")
        if p_value_mannwhitney_S3 < 0.05:
            print("解釈(パスa): 術後点眼薬の使用はΔBUTと有意に関連しており、媒介仮説の第一歩を支持します。")
        else:
            print("解釈(パスa): 術後点眼薬の使用はΔBUTと有意な関連が見られず、媒介仮説の第一歩は支持されませんでした。")
    else:
        print("\nSection 3: 術後点眼薬 vs ΔBUT の比較に必要なデータが不足しています。")
else:
    print("\n\n--- Section 3: スキップ (データ準備に失敗) ---")


# ================================================================================================
# Section 4 & 5: ΔBUT (M) と SPK改善 (Y) の関連性 (パス b)、および術後点眼薬 (X) の直接効果 (パス c')
# (前回のコードと同様のため、簡略化のためここでは省略しませんが、内容はほぼ同じです)
# 目的: ロジスティック回帰を用いて、SPK改善に対するΔBUTの効果(パスb)と、ΔBUT調整後の術後点眼の効果(直接効果c')を評価
# ------------------------------------------------------------------------------------------------
# SPK改善モデルのためのデータ準備 (df_spk_pre1 を使用)
df_mediation_analysis_S45 = pd.DataFrame() # 初期化
if not df_spk_pre1.empty and 'improved' in df_spk_pre1.columns:
    cols_for_dropna_S45 = ["improved", "eye_post", "ΔBUT"] + covariates
    df_mediation_analysis_S45 = df_spk_pre1.dropna(subset=cols_for_dropna_S45).copy()

if not df_mediation_analysis_S45.empty:
    print("\n\n--- Section 4 & 5: ΔBUT (M) & 術後点眼 (X) vs SPK改善 (Y) (媒介分析パスb, c'の準備) ---")
    y_logistic_S45 = df_mediation_analysis_S45["improved"]
    X_logistic_vars_S45 = ["eye_post", "ΔBUT"] + covariates
    X_logistic_S45 = df_mediation_analysis_S45[X_logistic_vars_S45]
    X_logistic_S45 = sm.add_constant(X_logistic_S45)

    if len(y_logistic_S45.unique()) < 2 :
        print("目的変数 'improved' の値が1種類しかないため、ロジスティック回帰を実行できません。")
    elif len(X_logistic_S45) < len(X_logistic_S45.columns):
        print("サンプル数が変数よりも少ないため、ロジスティック回帰を実行できません。")
    else:
        try:
            logit_model_S45 = sm.Logit(y_logistic_S45, X_logistic_S45)
            result_logistic_S45 = logit_model_S45.fit(disp=True)
            print("\nロジスティック回帰分析の結果 (SPK改善モデル):")
            params_S45 = result_logistic_S45.params
            conf_interval_S45 = result_logistic_S45.conf_int()
            conf_interval_S45.columns = ['CI 2.5%', 'CI 97.5%']
            odds_ratios_S45 = np.exp(params_S45)
            conf_interval_or_S45 = np.exp(conf_interval_S45)
            summary_table_S45 = pd.DataFrame({
                'Odds Ratio': odds_ratios_S45,
                'CI 2.5%': conf_interval_or_S45['CI 2.5%'],
                'CI 97.5%': conf_interval_or_S45['CI 97.5%'],
                'p_value': result_logistic_S45.pvalues
            })
            print(summary_table_S45)

            p_delta_but_S45 = np.nan # 初期化
            if 'ΔBUT' in summary_table_S45.index:
                or_delta_but_S45 = summary_table_S45.loc['ΔBUT', 'Odds Ratio']
                p_delta_but_S45 = summary_table_S45.loc['ΔBUT', 'p_value']
                print(f"\n解釈(パスb: ΔBUT -> SPK改善):")
                if p_delta_but_S45 < 0.05:
                    print(f"  ΔBUTはSPK改善と統計的に有意な関連があります (OR={or_delta_but_S45:.3f}, p={p_delta_but_S45:.4f})。媒介仮説のステップ M->Y を支持。")
                else:
                    print(f"  ΔBUTはSPK改善と統計的に有意な関連が見られませんでした (p={p_delta_but_S45:.4f})。")

            if 'eye_post' in summary_table_S45.index:
                or_eye_post_direct_S45 = summary_table_S45.loc['eye_post', 'Odds Ratio']
                p_eye_post_direct_S45 = summary_table_S45.loc['eye_post', 'p_value']
                print(f"\n解釈(直接効果c': eye_post -> SPK改善 [ΔBUT調整後]):")
                if p_eye_post_direct_S45 < 0.05:
                    print(f"  ΔBUT調整後も、術後点眼薬はSPK改善と有意な直接関連あり (OR={or_eye_post_direct_S45:.3f}, p={p_eye_post_direct_S45:.4f})。部分媒介の可能性。")
                else:
                    print(f"  ΔBUT調整後、術後点眼薬のSPK改善への直接効果は非有意 (p={p_eye_post_direct_S45:.4f})。")
                    if 'p_value_mannwhitney_S3' in locals() and p_value_mannwhitney_S3 < 0.05 and 'p_delta_but_S45' in locals() and p_delta_but_S45 < 0.05 :
                         print("    X->M と M->Y が共に有意で直接効果c'が非有意の場合、完全媒介の可能性が示唆される（ブートストラップで要検証）。")
        except Exception as e:
            print(f"\nロジスティック回帰モデルの推定中にエラーが発生しました: {e}")
else:
    print("\n\n--- Section 4 & 5: スキップ (媒介分析用のデータ準備に失敗、またはデータ不足) ---")

# ================================================================================================
# Section 6: ブートストラップ法による正式な媒介分析 (SPK改善モデル)
# ================================================================================================
# (前回のコードと同様のため、簡略化のためここでは省略しませんが、内容はほぼ同じです)
if not df_mediation_analysis_S45.empty: # Section 4&5 で準備したデータを使用
    print("\n\n--- Section 6: ブートストラップ法による媒介分析 (SPK改善モデル) ---")
    N_BOOTSTRAPS_S6 = 5000
    rng_S6 = np.random.default_rng(seed=123)
    indirect_effects_boot_S6 = []
    direct_effects_boot_S6 = []
    data_for_bootstrap_S6 = df_mediation_analysis_S45.copy()
    n_samples_S6 = len(data_for_bootstrap_S6)

    if n_samples_S6 == 0 or y_logistic_S45.nunique() < 2: # y_logistic_S45はSection 4&5で定義
        print("媒介分析に必要なデータが不足しているか、目的変数のバリエーションがありません。")
    else:
        print(f"ブートストラップを開始します (反復回数: {N_BOOTSTRAPS_S6}, サンプルサイズ: {n_samples_S6})...")
        for i in tqdm(range(N_BOOTSTRAPS_S6), desc="Bootstrapping Progress"):
            resampled_indices_S6 = rng_S6.choice(n_samples_S6, size=n_samples_S6, replace=True)
            df_resampled_S6 = data_for_bootstrap_S6.iloc[resampled_indices_S6]
            if df_resampled_S6["improved"].nunique() < 2:
                indirect_effects_boot_S6.append(np.nan)
                direct_effects_boot_S6.append(np.nan)
                continue
            try:
                Y_M_resampled_S6 = df_resampled_S6["ΔBUT"]
                X_M_vars_resampled_S6 = ["eye_post"] + covariates
                X_M_resampled_S6 = sm.add_constant(df_resampled_S6[X_M_vars_resampled_S6])
                model_M_resampled_S6 = sm.OLS(Y_M_resampled_S6, X_M_resampled_S6)
                results_M_resampled_S6 = model_M_resampled_S6.fit()
                coeff_a_S6 = results_M_resampled_S6.params["eye_post"]

                Y_Y_resampled_S6 = df_resampled_S6["improved"]
                X_Y_vars_resampled_S6 = ["eye_post", "ΔBUT"] + covariates
                X_Y_resampled_S6 = sm.add_constant(df_resampled_S6[X_Y_vars_resampled_S6])
                model_Y_resampled_S6 = sm.Logit(Y_Y_resampled_S6, X_Y_resampled_S6)
                results_Y_resampled_S6 = model_Y_resampled_S6.fit(disp=False, method='lbfgs', maxiter=100)
                if not results_Y_resampled_S6.mle_retvals['converged']:
                    results_Y_resampled_S6 = model_Y_resampled_S6.fit(disp=False, method='newton', maxiter=100)
                coeff_b_S6 = results_Y_resampled_S6.params["ΔBUT"]
                coeff_c_prime_S6 = results_Y_resampled_S6.params["eye_post"]
                indirect_effects_boot_S6.append(coeff_a_S6 * coeff_b_S6)
                direct_effects_boot_S6.append(coeff_c_prime_S6)
            except Exception:
                indirect_effects_boot_S6.append(np.nan)
                direct_effects_boot_S6.append(np.nan)

        indirect_effects_boot_S6 = np.array(indirect_effects_boot_S6)
        direct_effects_boot_S6 = np.array(direct_effects_boot_S6)
        indirect_effects_valid_S6 = indirect_effects_boot_S6[~np.isnan(indirect_effects_boot_S6)]
        direct_effects_valid_S6 = direct_effects_boot_S6[~np.isnan(direct_effects_boot_S6)]
        num_valid_iterations_S6 = len(indirect_effects_valid_S6)

        if num_valid_iterations_S6 > N_BOOTSTRAPS_S6 * 0.5 :
            try:
                Y_M_full_S6 = data_for_bootstrap_S6["ΔBUT"]
                X_M_full_S6 = sm.add_constant(data_for_bootstrap_S6[["eye_post"] + covariates])
                coeff_a_full_S6 = sm.OLS(Y_M_full_S6, X_M_full_S6).fit().params["eye_post"]

                Y_Y_full_S6 = data_for_bootstrap_S6["improved"]
                X_Y_full_S6 = sm.add_constant(data_for_bootstrap_S6[["eye_post", "ΔBUT"] + covariates])
                fit_Y_full_S6 = sm.Logit(Y_Y_full_S6, X_Y_full_S6).fit(disp=False, method='lbfgs', maxiter=100)
                if not fit_Y_full_S6.mle_retvals['converged']:
                    fit_Y_full_S6 = sm.Logit(Y_Y_full_S6, X_Y_full_S6).fit(disp=False, method='newton', maxiter=100)
                coeff_b_full_S6 = fit_Y_full_S6.params["ΔBUT"]
                coeff_c_prime_full_S6 = fit_Y_full_S6.params["eye_post"]
                point_estimate_indirect_S6 = coeff_a_full_S6 * coeff_b_full_S6
                point_estimate_direct_S6 = coeff_c_prime_full_S6
                ci_indirect_S6 = np.percentile(indirect_effects_valid_S6, [2.5, 97.5])
                ci_direct_S6 = np.percentile(direct_effects_valid_S6, [2.5, 97.5])

                print(f"\nブートストラップ法による媒介分析結果 ({num_valid_iterations_S6}/{N_BOOTSTRAPS_S6} 回の有効な反復):")
                print(f"間接効果 (a*b) 点推定値 (log-odds): {point_estimate_indirect_S6:.4f}, 95%CI: [{ci_indirect_S6[0]:.4f}, {ci_indirect_S6[1]:.4f}]")
                print(f"直接効果 (c') 点推定値 (log-odds): {point_estimate_direct_S6:.4f}, 95%CI: [{ci_direct_S6[0]:.4f}, {ci_direct_S6[1]:.4f}]")

                print("\n解釈 (媒介効果):")
                if ci_indirect_S6[0] * ci_indirect_S6[1] > 0:
                    print("  間接効果 (術後点眼 -> ΔBUT -> SPK改善) は統計的に有意。")
                    if ci_direct_S6[0] * ci_direct_S6[1] > 0: print("  直接効果も有意であり「部分媒介」。")
                    else: print("  直接効果は非有意であり「完全媒介」の可能性が高い。")
                else:
                    print("  間接効果 (術後点眼 -> ΔBUT -> SPK改善) は統計的に非有意 (95% CIが0を跨ぐ)。")
                    if ci_direct_S6[0] * ci_direct_S6[1] > 0: print("  ただし、直接効果は有意であり、ΔBUT以外の経路での術後点眼効果の可能性あり。")
                    else: print("  また、直接効果も非有意であり、術後点眼のSPK改善への影響は今回の分析では明確には確認できず。")
            except Exception as e:
                print(f"ブートストラップ結果集計/フルデータモデル推定中にエラー: {e}")
        else:
            print(f"ブートストラップ有効反復数不足 ({num_valid_iterations_S6}/{N_BOOTSTRAPS_S6})。")
else:
    print("\n\n--- Section 6: スキップ (媒介分析用のデータ準備に失敗、またはデータ不足) ---")

# =========================================
# Section 7: 全体的な結論と考察
# =========================================
print("\n\n--- Section 7: 全体的な結論と考察 ---")
print("このセクションでは、上記Section 2から6までの結果を総合的に解釈し、結論を記述します。")
print("考慮すべき点:\n")
print("1. Fisher検定による点眼薬とSPK変化（改善・予防）の直接的な関連はどうか？ (Section 2)")
print("2. 媒介分析の各パス (a: X->M, b: M->Y|X, c': X->Y|M) はどうだったか？ (Section 3, 4&5)")
print("3. ブートストラップによる間接効果 (a*b) の95%信頼区間は0を跨いでいたか？ (Section 6)")
print("\nこれらの結果を基に、当初の仮説（点眼の直接効果、およびΔBUTを介した媒介効果）が")
print("どの程度統計的に支持されたか、あるいはされなかったかを評価してください。")
print("最終的な解釈は、これらの統計的証拠と臨床的知見を合わせて行うことが重要です。")

if df_all_original.empty or (df_spk_pre1.empty and df_spk_pre0.empty):
    print("\n注意: データファイルの読み込みまたは前処理に問題があったため、解析が十分に実行されていません。")

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: overflow encountered in exp
  result = func(self.values, **kwargs)


ファイル '/content/drive/Shareddrives/岩崎Dr_IgG4 deulk/眼瞼下垂ドライアイ/眼瞼下垂★ドライアイ_kit.xlsx' を正常に読み込みました。

データ準備完了。
SPK改善モデルの対象症例数 (SPK pre=1): 96
  うちSPK改善 (improved=1) した症例数: 26
SPK悪化/予防モデルの対象症例数 (SPK pre=0): 157
  うちSPK悪化 (worsened=1) した症例数: 45


--- Section 2: 点眼薬使用とSPK変化の直接的関連 (Fisher検定) ---

--- 2.1 SPK改善 (術前SPK(+)症例) と点眼薬の関連 ---

eye_pre と SPK改善の関連:
               Not Improved (SPK post=1)  Improved (SPK post=0)
eye_pre (No)                          35                     19
eye_pre (Yes)                         32                      7
  Odds Ratio: 0.403, Fisher's Exact p-value: 0.1005
  解釈: eye_pre の使用とSPK改善には統計的に有意な関連は見られませんでした。

eye_post と SPK改善の関連:
                Not Improved (SPK post=1)  Improved (SPK post=0)
eye_post (No)                          31                     19
eye_post (Yes)                         36                      7
  Odds Ratio: 0.317, Fisher's Exact p-value: 0.0225
  解釈: eye_post の使用とSPK改善には統計的に有意な関連が見られます。

--- 2.2 SPK悪化/予防 (術前SPK(-)症例) と点眼薬の関連 ---

eye_p

ストリーミング出力は最後の 5000 行に切り捨てられました。
Bootstrapping Progress:  21%|██        | 1049/5000 [00:35<02:09, 30.42it/s]/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
Bootstrapping Progress:  21%|██        | 1053/5000 [00:35<02:10, 30.20it/s]/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  


ブートストラップ法による媒介分析結果 (4551/5000 回の有効な反復):
間接効果 (a*b) 点推定値 (log-odds): 0.1087, 95%CI: [-2.0402, 3.0500]
直接効果 (c') 点推定値 (log-odds): -10.6383, 95%CI: [-100.3770, -0.4790]

解釈 (媒介効果):
  間接効果 (術後点眼 -> ΔBUT -> SPK改善) は統計的に非有意 (95% CIが0を跨ぐ)。
  ただし、直接効果は有意であり、ΔBUT以外の経路での術後点眼効果の可能性あり。


--- Section 7: 全体的な結論と考察 ---
このセクションでは、上記Section 2から6までの結果を総合的に解釈し、結論を記述します。
考慮すべき点:

1. Fisher検定による点眼薬とSPK変化（改善・予防）の直接的な関連はどうか？ (Section 2)
2. 媒介分析の各パス (a: X->M, b: M->Y|X, c': X->Y|M) はどうだったか？ (Section 3, 4&5)
3. ブートストラップによる間接効果 (a*b) の95%信頼区間は0を跨いでいたか？ (Section 6)

これらの結果を基に、当初の仮説（点眼の直接効果、およびΔBUTを介した媒介効果）が
どの程度統計的に支持されたか、あるいはされなかったかを評価してください。
最終的な解釈は、これらの統計的証拠と臨床的知見を合わせて行うことが重要です。
